# Load environment

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from bs4 import BeautifulSoup # Used for separating text and code
from nltk.stem import SnowballStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.neighbors import NearestNeighbors
import re
from bs4 import BeautifulSoup

from keras.layers import Input, Concatenate, Dense, Reshape, Flatten
from keras.models import Model
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

question_tag_df = pd.read_csv("/content/drive/MyDrive/G1T3 IS450/Dataset/FINAL.csv", converters={'code_snippets': literal_eval}, encoding='ISO-8859-1')
answers_df = pd.read_csv("/content/drive/MyDrive/G1T3 IS450/Dataset/shortend_answers_final.csv", encoding='ISO-8859-1')


Mounted at /content/drive


In [3]:
question_tag_df

,Id,CreationDate,Score,Title,Body,Tag,code_snippets,body_words,body_words_preprocessed_task1,title_preprocessed_task1,body_words_preprocessed_task2,title_preprocessed_task2,weight_title_body_preprocessed_task2,weight_title_body_preprocessed_task1
0,19280,2008-08-21T03:20:05Z,16,Code to ask yes/no question in javascript,<p>I could only find the function <code>confir...,['javascript'],[],I could only find the function confirm() that ...,could function confirm ( ) ok/cancel button . ...,code yes/no question javascript,i could onli find the function confirm ( ) tha...,code to ask yes/no question in javascript,code to ask yes/no question in javascript code...,code yes/no question javascript code yes/no qu...
1,19970,2008-08-21T13:58:25Z,2,Best way to keep an ordered list of windows (f...,<p>What is the best way to manage a list of wi...,['javascript'],[],What is the best way to manage a list of windo...,best way list window ( order ) abl next window...,best way order list window ( most-rec ) ?,what is the best way to manag a list of window...,best way to keep an order list of window ( fro...,best way to keep an order list of window ( fro...,best way order list window ( most-rec ) ? best...
2,22720,2008-08-22T15:58:13Z,3,Configure a Java Socket to fail-fast on discon...,<p>I have a listening port on my server that I...,"['java', 'exception', 'sockets', 'networking']",[],I have a listening port on my server that I'm ...,"port server java class socket interfac , i.e.i...",configur java socket fail-fast disconnect ?,i have a listen port on my server that i 'm co...,configur a java socket to fail-fast on disconn...,configur a java socket to fail-fast on disconn...,configur java socket fail-fast disconnect ? co...
3,29680,2008-08-27T07:40:01Z,5,Suppress NTLM dialog box after unauthorized re...,"<p>In a recent sharepoint project, I implement...","['asp.net', 'javascript', 'sharepoint']",[function Login() { var request = GetRequest...,"In a recent sharepoint project, I implemented ...","recent sharepoint project , authent webpart nt...",suppress dialog box unauthor request,"in a recent sharepoint project , i implement a...",suppress ntlm dialog box after unauthor request,suppress ntlm dialog box after unauthor reques...,suppress dialog box unauthor request suppress ...
4,30540,2008-08-27T16:02:00Z,6,What does this javascript error mean? Permissi...,<p>This error just started popping up all over...,"['javascript', 'flash']",[],This error just started popping up all over ou...,error site.permiss method googl post crossdoma...,javascript error mean ? permiss method,this error just start pop up all over our site...,what doe this javascript error mean ? permiss ...,what doe this javascript error mean ? permiss ...,javascript error mean ? permiss method javascr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62006,40139580,2016-10-19T19:05:18Z,0,Wordpress Plugin error. Backup Buddy,<p>I am having a error when using <code>Backup...,"['javascript', 'wordpress']",[],I am having a error when using BackupBuddy plu...,error backupbuddi plugin wordpress . error any...,wordpress plugin error . backup buddi,i am have a error when use backupbuddi plugin ...,wordpress plugin error . backup buddi,wordpress plugin error . backup buddi wordpres...,wordpress plugin error . backup buddi wordpres...
62007,40139900,2016-10-19T19:23:29Z,-1,"Shopify not loading JS script, although loadin...",<p>So I'm trying to figure out what I thought ...,"['javascript', 'jquery']",[],So I'm trying to figure out what I thought was...,thought relat simpl issu . follow : jqueri ( l...,"shopifi js script , load consol work",so i 'm tri to figur out what i thought was a ...,"shopifi not load js script , although load in ...","shopifi not load js script , although load in ...","shopifi js script , load consol work shopifi j..."
62008,40140970,2016-10-19T20:29:16Z,-1,autoplay the contents within a div with jquery,<p>I have some contents within different divs ...,"['javas

In [4]:
df = question_tag_df[['Id','Score','body_words','Title','weight_title_body_preprocessed_task2']]

In [5]:
def preprocess_data(df, body_column_name, code_column_name, body_words_column_name):
    df[code_column_name] = ''
    df[body_words_column_name] = ''


    for index, row in df.iterrows():
        # Removing all the escape sequences
        row[body_column_name] = re.sub(r'[\n\t\r\f]', '', row[body_column_name]) # Not sure if you want us to remove it

        # Split the body into code_snippets and body_words
        soup = BeautifulSoup(row[body_column_name], 'html.parser')

        code_snippets = []
        text_without_code = ''

        # Extract code snippets from <pre><code></pre></code> tags
        for pre in soup.find_all('pre'):
            code_snippets.extend(code.get_text() for code in pre.find_all('code'))
            pre.decompose()  # Remove <pre> tags and their contents

        # Extract text content without code snippets
        text_without_code = soup.get_text()

        df.at[index, code_column_name] = code_snippets
        df.at[index, body_words_column_name] = text_without_code

    return df


# Add presence/ absent of code
def presence_of_code_answer(code_snippets):
    if len(code_snippets) > 0:
        return 1
    else:
        return 0

# Convert the scores into bins
def map_to_category(score):
    if score <= q1:
        return 0  # Low
    elif score <= q2:
        return 1  # Medium
    else:
        return 2  # High


In [6]:
df

,Id,Score,body_words,Title,weight_title_body_preprocessed_task2
0,19280,16,I could only find the function confirm() that ...,Code to ask yes/no question in javascript,code to ask yes/no question in javascript code...
1,19970,2,What is the best way to manage a list of windo...,Best way to keep an ordered list of windows (f...,best way to keep an order list of window ( fro...
2,22720,3,I have a listening port on my server that I'm ...,Configure a Java Socket to fail-fast on discon...,configur a java socket to fail-fast on disconn...
3,29680,5,"In a recent sharepoint project, I implemented ...",Suppress NTLM dialog box after unauthorized re...,suppress ntlm dialog box after unauthor reques...
4,30540,6,This error just started popping up all over ou...,What does this javascript error mean? Permissi...,what doe this javascript error mean ? permiss ...
...,...,...,...,...,...
62006,40139580,0,I am having a error when using BackupBuddy plu...,Wordpress Plugin error. Backup Buddy,wordpress plugin error . backup buddi wordpres...
62007,40139900,-1,So I'm trying to figure out what I thought was...,"Shopify not loading JS script, although loadin...","shopifi not load js script , although load in ..."
62008,40140970,-1,I have some contents within different divs tha...,autoplay the contents within a div with jquery,autoplay the content within a div with jqueri ...
62009,40141920,0,I'm working on a project and I'm trying to cou...,InputStream reading file and counting lines/words,inputstream read file and count lines/word inp...


In [7]:
def preprocessing_task2(text):
    text = text.lower()
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

In [8]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation marks
    text = re.sub(r'[' + re.escape(string.punctuation) + ']', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    # Filter out short tokens
    min_token_length = 3
    tokens = [token for token in tokens if len(token) >= min_token_length]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

# Apply preprocessing to each document
df['preprocessed'] = df['weight_title_body_preprocessed_task2'].apply(preprocess_text)

# Use last 10 rows for user queries
df_queries = df[['preprocessed']].iloc[-10:].reset_index(drop=True)

# extract the last 10 titles from df
last_10_titles = df['Title'].iloc[-10:].reset_index(drop=True)
last_10_body = df['body_words'].iloc[-10:].reset_index(drop=True)

# Add the titles to df_queries
df_queries['Title'] = last_10_titles
df_queries['body_words'] = last_10_body

# Exclude the last 10 rows for the main DataFrame
df = df.iloc[:-10]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-8-31b299b27412>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed'] = df['weight_title_body_preprocessed_task2'].apply(preprocess_text)


In [9]:
import sys
import pandas as pd

# Set the maximum number of rows and columns displayed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Set the maximum width of columns displayed
pd.set_option('display.max_colwidth', None)

# Set the maximum output display
pd.set_option('display.max_info_columns', sys.maxsize)
pd.set_option('display.width', 1000)


# demo

In [10]:
# Using bi-grams or tri-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # include both single words and bi-grams

tfidf_matrix = vectorizer.fit_transform(df['preprocessed'])

# Transform the preprocessed text in df_queries to a TF-IDF matrix
tfidf_matrix_queries = vectorizer.transform(df_queries['preprocessed'])

# K-Means

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np

# Fit K-Means on the combined TF-IDF vectors of both original data and  queries
num_clusters = 16
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# Get the cluster labels for the original data
original_data_labels = kmeans.labels_[:-len(df_queries)]

# find the top 5 questions within the same cluster for each  query
for i, query_tfidf_vector in enumerate(tfidf_matrix_queries):
    # Get the corresponding cluster for the query
    query_cluster_label = kmeans.labels_[-len(df_queries) + i]

    # Find indices of the data points in the same cluster
    same_cluster_indices = np.where(original_data_labels == query_cluster_label)[0]

    # Calculate cosine similarity between the query and all points in the same cluster
    cosine_similarities = cosine_similarity(query_tfidf_vector, tfidf_matrix[same_cluster_indices])

    # Get the top 5 indices of the most similar questions
    top5_indices = np.argsort(cosine_similarities[0])[-5:][::-1]

    # Retrieve the original data indices of the top 5 similar questions
    top5_data_indices = same_cluster_indices[top5_indices]

    # print
    print(f"\nThe Query '{df_queries['Title'].iloc[i]}' falls into cluster {query_cluster_label}")
    print("Top 5 most similar questions within the cluster:")

    for data_index in top5_data_indices:
        title = question_tag_df.iloc[data_index]['Title']
        body_words = question_tag_df.iloc[data_index]['body_words']
        print(f"Title: {title}")
        print(f"Body Words: {body_words}")
        print('\n')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



The Query 'How to keep nav menu highlighted after click using js in sidebar' falls into cluster 4
Top 5 most similar questions within the cluster:
Title: Keep active menu item highlighted
Body Words: How would I make it so that the item that is clicked in the menu, stays highlighted blue. So basically the active menu item.FiddleI've tried using css active, but im thinking I need javascript or something.


Title: Fixing a CSS problem for my JavaScript powered nav menu
Body Words: Hey, I have a JavaScript powered navigation bar used for the primary navigation on my website, www.marioplanet.com.I am having some trouble when I try to adjust the width of my nav bar to be flush with the rest of my page, and it appears to throw the height off for some reason.Any suggestions would be helpful and greatly appreciated!


Title: Javascript menu conflict
Body Words: I'm having a Javascript conflict. I have a vertical javascript left nav menu and a horizontal javascript time zone menu. Working code

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Reduce the dimensionality of the TF-IDF vectors
pca = PCA(n_components=2, random_state=42)
reduced_vectors = pca.fit_transform(tfidf_matrix.toarray())

# scatter plot
plt.figure(figsize=(10, 8))

for i in range(num_clusters):
    cluster_indices = np.where(kmeans.labels_ == i)[0]
    cluster_points = reduced_vectors[cluster_indices]

    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {i}', alpha=0.6)

plt.title('Clusters visualization with PCA')
plt.xlabel('PCA Feature 1')
plt.ylabel('PCA Feature 2')
plt.legend()
plt.show()


## KNN


In [13]:

knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(tfidf_matrix)
top_questions = []

for i, query_tfidf_vector in enumerate(tfidf_matrix_queries):
    distances, indices = knn.kneighbors(query_tfidf_vector)

    # Print out the original query
    print(f"Query: {df_queries['Title'].iloc[i]}")
    print(f"Query_body: {df_queries['body_words'].iloc[i]}")
    print("----Nearest neighbors:-----")

    # Loop through the indices to print the nearest neighbors
    for index in indices.flatten():
        title = question_tag_df.loc[index, 'Title']
        body_words = question_tag_df.loc[index, 'body_words']
        print(f"Title: {title}")
        print(f"Body Words: {body_words}\n")
    print('\n')
df_qns = question_tag_df[question_tag_df["Title"].isin(top_questions)]
top_answers_df = pd.merge(df_qns,
              answers_df,
              how='inner',
              left_on='Id',
              right_on='ParentId')

top_answers_df.rename(columns={"Id_x":"question_id","code_snippets":"code_snippets_question","Id_y":"answer_id","Score_y":"score_answer","Body_y":"body_answer","Body_x":"body_words_question"},inplace =True)
top_answers_df.drop(columns=["CreationDate_x","Score_x","Tag","body_words_preprocessed_task1","title_preprocessed_task1","body_words_preprocessed_task2","title_preprocessed_task2","weight_title_body_preprocessed_task1","OwnerUserId","CreationDate_y","ParentId"],inplace=True)



Query: How to keep nav menu highlighted after click using js in sidebar
Query_body: I have created a nav menu in Wordpress and placed it in sidebar.php in my child theme.  My nav menu is in the correct location and functions and looks as it should with the exception of the JS, which I have tried to get right but seem to be failing. Each menu item takes you to a different page. I want the menu item to stay highlighted to show which page you are on once you have linked to that page. I have used CSS to highlight the li using :hover. I cant work out how or where to place the js to keep it the menu highlighted. Is my JS correct?Where do I place it? directly in sidebar.php under the html? or somewhere else?Thank you!
----Nearest neighbors:-----
Title: jQuery smooth scrolling and menu highlight
Body Words: I want to create jQuery smooth scrolling and highlight menu using js and cssMy menu html structureSite html structure also js function And new class. I want to use it to highlight current m

# Evaluation for K-Means (Silhouette Coefficient)

In [14]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Fit K-Means
kmeans = KMeans(n_clusters= num_clusters, random_state=42)
kmeans_labels = kmeans.fit_predict(tfidf_matrix)

# Calculate Silhouette Coefficient for the clusters
silhouette_avg = silhouette_score(tfidf_matrix, kmeans_labels)
print(f'Silhouette Coefficient: {silhouette_avg:.4f}')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette Coefficient: 0.0039
